# Финальный пайплайн обучения и теста


In [ ]:
!pip install vesuvius
!vesuvius.accept_terms --yes

In [ ]:
import vesuvius
from vesuvius import Volume
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.optim as optim
from torch.nn.utils import clip_grad_norm_

import numpy as np
from tqdm import tqdm
import gc

In [ ]:
class VolumetricDataset(Dataset):
    def __init__(self, volume, label, tile_size, stride, validation_zone, valid=False):
        """
        Initialize the dataset with the volume and label.
        
        volume (np.ndarray): The volumetric image of shape (Z, Y, X).
        label (np.ndarray): The 2D label of shape (Y, X).
        tile_size (int): The size of the tiles to extract along the Y and X dimensions.
        stride (int): The stride for extracting tiles along the Y and X dimensions.
        """
        
        self.volume = volume
        self.label = label
        self.tile_size = tile_size
        self.stride = stride
        self.validation_zone = validation_zone
        self.valid = valid
        self.tiles, self.labels, self.corners = self.extract_tiles()

    def extract_tiles(self):
        """
        Extract 3D tiles from the volume and corresponding 2D labels.

        Returns:
            tiles (list): A list of 3D tiles.
            labels (list): A list of 2D labels.
        """
        Z, Y, X = self.volume.shape
        tiles = []
        labels = []
        corners = []
        # generate 3D tiles by moving along the Y and X axes
        for y in range(0, Y - self.tile_size + 1, self.stride):
            for x in range(0, X - self.tile_size + 1, self.stride):
                if self.valid is False:
                    if (y + self.tile_size < self.validation_zone[0]) or (y > self.validation_zone[1]):
                        if (x + self.tile_size < self.validation_zone[2]) or (x > self.validation_zone[3]):
                            tile = self.volume[:, y:y + self.tile_size, x:x + self.tile_size]
                            label_tile = self.label[y:y + self.tile_size, x:x + self.tile_size]
                            if np.sum(label_tile)/self.tile_size**2 > 0.05: # at least 5% of ink
                                tiles.append(tile)
                                labels.append(label_tile)
                                corners.append([y,x])
                else:
                    if (y >= self.validation_zone[0]) and (y + self.tile_size <= self.validation_zone[1]):
                        if (x >= self.validation_zone[2]) and (x + self.tile_size <= self.validation_zone[3]):
                            tile = self.volume[:, y:y + self.tile_size, x:x + self.tile_size]
                            label_tile = self.label[y:y + self.tile_size, x:x + self.tile_size]
                            tiles.append(tile)
                            labels.append(label_tile)
                            corners.append([y,x])

        return tiles, labels, corners

    def __len__(self):
        return len(self.tiles)

    def __getitem__(self, idx):
        tile = self.tiles[idx] 
        label = self.labels[idx]
        corners = self.corners[idx]
        
        tile = torch.tensor(tile, dtype=torch.float32).unsqueeze(0)
        label = torch.tensor(label, dtype=torch.float32).unsqueeze(0)
        corners = torch.tensor(corners, dtype=torch.int).unsqueeze(0)
        return tile, label, corners

In [ ]:
def fractal_dimension(binary):
    '''
    Calculate fractal dimension of binary image ln(N) / ln(1/r) using box counting method.
    '''
    device = binary.device
    batch_size, blocks_num, height, width = binary.shape
    min_dim = min(height, width)
    # square sizes to use box counting method
    scales = torch.tensor([1,2,4,6,8,10], device=device)
    scales = torch.unique(scales)
    scales = scales[scales <= min_dim]
    counts = []

    binary_flat = binary.view(-1, H, W)
    cnt_flat = binary_flat.size(0)
    for scale in scales:
        if scale < 1:
            continue

        # add zero padding to use box counting method
        pad_h = (scale - (H % scale)) % scale
        pad_w = (scale - (W % scale)) % scale
        padded_binary = F.pad(binary_flat, (0, pad_w, 0, pad_h), value=False)

        # split image into small squares to use box counting method
        unfolded = padded_binary.unfold(1, scale, scale).unfold(2, scale, scale)
        count = unfolded.any(dim=-1).any(dim=-1).sum(dim=(1,2))
        counts.append(count)

    if len(counts) < 2:
        return torch.zeros(batch_size, blocks_num, device=device)
    
    log_scales = torch.log(1 / scales.float() + 1e-8) # log(1/r)
    log_counts = torch.log(torch.stack(counts, dim=1).float() + 1e-8) # log(N)

    # here we prepare matricies to get the slope of the line that best fits the
    # 2D points of the form (ln(N), ln(1/r))
    # that way we will get fractal dimensions
    X = torch.stack([log_scales, torch.ones_like(log_scales)], dim=1) 
    X = X.repeat(cnt_flat, 1, 1)
    Y = log_counts.unsqueeze(-1)

    # get the slope of the line that best fits the 2D points of the form (ln(N), ln(1/r)) 
    coeffs = torch.linalg.lstsq(X, Y).solution.squeeze(-1)

    # FD is first coefficient (slope of the line)
    fd = coeffs[..., 0]
    return fd


def calc_fractal_features(input_volume, windows = [2,4,8], thresholds=None, add_image_channel = False):
    '''
    Calculate fractal features of image with certain sizes of window 
    and certain thresholds to make image binary (binary = (image > image.mean() * threshold)).

    Also we could add original image to those features.
    '''
    device = input_volume.device
    batch_size, channels, z, y, x = input_volume.shape # channels = 1

    volume = input_volume.view(batch_size*channels, z, y, x)

    max_window_size = min(y, x) // 2
    windows = torch.tensor(windows, device=device)
    windows = windows[windows <= max_window_size]

    if thresholds is None:
        thresholds = [1.]
    avg = volume.mean(dim=(2,3))
    new_thresholds = []
    for threshold in thresholds:
        new_thresholds.append(threshold * avg)
    thresholds = new_thresholds

    num_features = len(thresholds) * len(windows)
    if add_image_channel:
        num_features += 1
    
    features = torch.zeros(batch_size, num_features, z, y, x, device=device)
    cur_channel = 0
    
    if add_image_channel:
        features[:, 0] = volume
        cur_channel += 1

    for t_idx, threshold in enumerate(thresholds):
        
        # if we want to add relative to mean thresholds
        if isinstance(threshold, torch.Tensor):
            threshold = threshold.view(batch_size, z, 1, 1)
        
        binary = volume > threshold
        for window in windows:
            # Add padding to binary image to calculate fractal feautures
            pad_y = (window - (y % window)) % window
            pad_x = (window - (x % window)) % window
            padded_binary = F.pad(binary, (0, pad_x, 0, pad_y), value=False)

            # split image into small squares to calculate fractal feautures for them
            unfolded = padded_binary.unfold(2, window, window).unfold(3, window, window)
            _, _, y_unf, x_unf, _, _ = unfolded.shape

            # calculate fractal dimension for all squares
            fd = fractal_dimension(
                unfolded.reshape(batch_size*z, y_unf*x_unf, window, window)
            ).reshape(batch_size, z, y_unf, x_unf)

            # interpolate to original shape of image
            fd_upsampled = F.interpolate(
                fd,
                size=(y, x),
                mode='nearest'
            )

            features[:, cur_channel] = fd_upsampled.squeeze(1)
            cur_channel += 1

    return features

In [ ]:
class UNet(nn.Module):
    '''
    UNet model without fractal features.
    '''
    def __init__(self, z=16, y = 256, x = 256):
        super(UNet, self).__init__()
        self.y = y
        self.x = x
        self.z = z
        self.in_channels = 1

        # input layers
        self.conv3d = nn.Conv3d(self.in_channels, 128, kernel_size=3, padding=1)
        self.attn = nn.Sequential(
            nn.Conv3d(128, 1, kernel_size=1),
            nn.Softmax(dim=2) # softmax over z-dimension (depth)
        )

        self.in_conv = nn.Conv2d(128, 128, kernel_size=3, padding=1)

        # contracting path
        self.enc_conv1 = self.double_conv(128, 128)
        self.enc_conv2 = self.double_conv(128, 256)
        self.enc_conv3 = self.double_conv(256, 512)
        self.enc_conv4 = self.double_conv(512, 1024)

        # expansive path
        self.up_trans1 = self.up_conv(1024, 512)
        self.dec_conv1 = self.double_conv(1024, 512)
        self.up_trans2 = self.up_conv(512, 256)
        self.dec_conv2 = self.double_conv(512, 256)
        self.up_trans3 = self.up_conv(256, 128)
        self.dec_conv3 = self.double_conv(256, 128)

        # final output
        self.out_conv = nn.Conv2d(128, 1, kernel_size=1)

    def forward(self, x):
        x = self.conv3d(x)
        attn = self.attn(x)
        x = torch.sum(x * attn, dim=2)

        x = self.in_conv(x)

        # contracting path
        x1 = self.enc_conv1(x)
        x2 = self.enc_conv2(F.max_pool2d(x1, kernel_size=2))
        x3 = self.enc_conv3(F.max_pool2d(x2, kernel_size=2))
        x4 = self.enc_conv4(F.max_pool2d(x3, kernel_size=2))

        # expansive path
        x = self.up_trans1(x4)
        x = torch.cat([x, x3], dim=1)
        x = self.dec_conv1(x)

        x = self.up_trans2(x)
        x = torch.cat([x, x2], dim=1)
        x = self.dec_conv2(x)

        x = self.up_trans3(x)
        x = torch.cat([x, x1], dim=1)
        x = self.dec_conv3(x)

        x = self.out_conv(x)

        return x

    def double_conv(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def up_conv(self, in_channels, out_channels):
        return nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

def initialize_weights(model):
    '''
    Initialize weights.
    '''
    for m in model.modules():
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Conv3d):
            nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.ConvTranspose2d):
            nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm3d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)

In [ ]:
train_segment_ids=[
    20230520175435,
    20230522181603,
    20230522215721,
    20230530164535,
    20230530172803,
    20230530212931,
    20230531121653,
    20230531193658,
    20230601193301,
    20230611014200,
    20230620230617,
    20230620230619,
    20230701020044,
    20230820203112,
    20230826170124,
    20230901184804,
    20230902141231,
    20230903193206,
    20230904020426,
    20230904135535,
    20230905134255,
    20230909121925,
    20231001164029,
    20231004222109,
    20231012085431,
    20231012184420, # big
]
    

val_segment_id = 20230827161847

tile_size = 256 
stride = 128  # stride for moving the tile in the YX dimension
batch_size = 4
z_depth = 16 # thickness of the tile

val_segment = Volume(val_segment_id, normalize=True)
validation_rect = [200,5600,1000,4600] # zone with ink
valid_dataset = VolumetricDataset(val_segment[(32 - z_depth//2):(32+z_depth//2),200:5600,1000:4600,0], val_segment.inklabel[200:5600,1000:4600]/255, tile_size, stride, validation_zone=validation_rect, valid=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
!pip install wandb
import wandb

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(z = z_depth, y = tile_size, x = tile_size)
initialize_weights(model)
model = model.to(device)
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
NUM_EPOCHS = 50
scheduler = CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

In [ ]:
from tqdm import tqdm
import time
clip_value = 10.0

scaler = GradScaler()
model.train()
no_improvement = 0
best_score = 1e9
start_time = time.time()
logs = open("logs.txt", "w")
for epoch in tqdm(range(NUM_EPOCHS)):
    running_loss = 0.0
    iters = 0
    for seg_id in train_segment_ids[:]:
        print(f"Epoch {epoch}, segment: {seg_id}")
        
        # initialize segment at each epoch to avoid memory problems
        segment = Volume(seg_id, normalize=True)
        dataset = VolumetricDataset(
            segment[(32 - z_depth//2):(32+z_depth//2),:,:,0],
            segment.inklabel[:,:]/255, tile_size, stride,
            validation_zone=[0,0,0,0], valid=False)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        
        model.train()

        for batch_tiles, batch_labels, _ in tqdm(dataloader, position=0, leave=False):
            batch_tiles, batch_labels = batch_tiles.to(device), batch_labels.float().to(device)
            optimizer.zero_grad()
    
            with autocast(device_type=device.type):
                outputs = model(batch_tiles)
                loss = criterion(outputs, batch_labels)

            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            clip_grad_norm_(model.parameters(), clip_value)

            scaler.step(optimizer)
            scaler.update()
            
            running_loss += loss.item()
            iters += 1
        
        print(f"Loss {running_loss/iters}")
        segment = None
        dataset = None
        dataloader = None
        gc.collect()
        torch.cuda.empty_cache()
        
    scheduler.step()
    
    print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], running loss: {running_loss/iters}')
    model.eval()
    
    val_score = 0
    with torch.no_grad():
        for batch_tiles, batch_labels, corners in valid_dataloader:
            batch_tiles, batch_labels = batch_tiles.to(device), batch_labels.float().to(device)
    
            with autocast(device_type=device.type):
                outputs = model(batch_tiles)

            val_score += criterion(outputs, batch_labels)
    val_score /= len(valid_dataloader)
    print(f"Val loss: {val_score}")

    if best_score > val_score:
        best_score = val_score
        torch.save(model.state_dict(), f'best_model_epoch_{epoch}_{val_score}.pth')
    else:
        torch.save(model.state_dict(), f'model_epoch_{epoch}_{val_score}.pth')
    logs.write(f"{epoch} {running_loss/iters} {val_score}\n")
        
    if running_loss/iters < 0.08 or time.time() - start_time > 18000:
        print(f"Final loss {running_loss/iters}")
        break
print("Training completed.")
logs.close()

In [ ]:
# initialize predictions
letter_predictions = np.zeros_like(val_segment.inklabel[200:5600, 1000:4600], dtype=np.float32)
counter_predictions = np.zeros_like(val_segment.inklabel[200:5600, 1000:4600], dtype=np.float32)
loss = 0.0

model.eval()
with torch.no_grad():
    for batch_tiles, batch_labels, corners in valid_dataloader:
        batch_tiles, batch_labels = batch_tiles.to(device), batch_labels.float().to(device)

        with autocast(device_type=device.type):
            outputs = model(batch_tiles)

        loss += criterion(outputs, batch_labels)
        # apply sigmoid to get probabilities from logits
        predictions = torch.sigmoid(outputs)

        # update tiles predictions
        corners = corners.squeeze(1).cpu().numpy()
        for idx in range(corners.shape[0]):
            x_start, y_start = corners[idx, 0], corners[idx, 1]
            prediction_tile = predictions.cpu().numpy()[idx, 0] 
            letter_predictions[x_start:x_start + tile_size, y_start:y_start + tile_size] += prediction_tile
            counter_predictions[x_start:x_start + tile_size, y_start:y_start + tile_size] += 1

print(loss / len(valid_dataloader))

# avoid division by zero by setting any zero counts to 1
counter_predictions[counter_predictions == 0] = 1

# normalize the predictions by the counter values
letter_predictions /= counter_predictions

# plot ground truth and model predictions
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

ax = axes[0]
ax.imshow(val_segment.inklabel[200:5600, 1000:4600] / 255, cmap='gray')
ax.set_title('Ground Truth Label')
ax.axis('off')

ax = axes[1]
ax.imshow(letter_predictions, cmap='gray')
ax.set_title('Model Prediction')
ax.axis('off')

plt.savefig("final_res_UNet.png")
plt.show()

In [ ]:
torch.save(model.state_dict(), 'final_model_UNet.pth')

In [ ]:
# initialize predictions
letter_predictions = np.zeros_like(val_segment.inklabel[200:5600, 1000:4600], dtype=np.float32)
counter_predictions = np.zeros_like(val_segment.inklabel[200:5600, 1000:4600], dtype=np.float32)
loss = 0.0

model.eval()
with torch.no_grad():
    for batch_tiles, batch_labels, corners in valid_dataloader:
        batch_tiles, batch_labels = batch_tiles.to(device), batch_labels.float().to(device)

        with autocast(device_type=device.type):
            outputs = model(batch_tiles)

        loss += criterion(outputs, batch_labels)
        # apply sigmoid to get probabilities from logits
        predictions = torch.sigmoid(outputs)

        # update tiles predictions
        corners = corners.squeeze(1).cpu().numpy()
        for idx in range(corners.shape[0]):
            x_start, y_start = corners[idx, 0], corners[idx, 1]
            prediction_tile = predictions.cpu().numpy()[idx, 0] 
            letter_predictions[x_start:x_start + tile_size, y_start:y_start + tile_size] += prediction_tile
            counter_predictions[x_start:x_start + tile_size, y_start:y_start + tile_size] += 1

print(loss / len(valid_dataloader))

# avoid division by zero by setting any zero counts to 1
counter_predictions[counter_predictions == 0] = 1

# normalize the predictions by the counter values
letter_predictions /= counter_predictions

# plot ground truth and model predictions
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

ax = axes[0]
ax.imshow(val_segment.inklabel[200:5600, 1000:4600] / 255, cmap='gray')
ax.set_title('Ground Truth Label')
ax.axis('off')

ax = axes[1]
ax.imshow(letter_predictions, cmap='gray')
ax.set_title('Model Prediction')
ax.axis('off')

plt.savefig("best_res_UNet.png")
plt.show()